# Travelling Salesperson Problem solved using genetic algorithms

In [15]:
# Imports 
import numpy as np
import random
import pandas as pd
from geopy import distance,geocoders,Nominatim # to calculate distance on the surface
import folium
from datetime import datetime

In [16]:
# Parameters
n_cities = 10

n_population = 50

mutation_rate = 0.3

df = pd.read_csv('output.csv')

In [17]:
# Generating a list of coordenades representing each city
geolocator = geocoders.Nominatim(user_agent="my_email@myserver.com")
coordinates_list=[]
names_list = np.array(df['Street'].head(n_cities))
for city in names_list:
    location = geolocator.geocode(city)
    coordinates_list.append(list((location.latitude, location.longitude)))
    
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return distance.distance((a[0], a[1]), (b[0],b[1])).km

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

# cities_dict

## 1. Create the first population set
We randomly shuffle the cities N times where N=population_size

In [18]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
# population_set

## 2. Evaluate solutions fitness
The solutions are defined so that the first element on the list is the first city to visit, then the second, etc. and the last city is linked to the first.
The fitness function needs to compute the distance between subsequent cities.

In [19]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [20]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
# fitnes_list

# 3. Progenitors selection
I will select a new set of progenitors using the Roulette Wheel Selection. Generates a list of progenitor pairs where N= len(population_set) but at each position there are two solutions to merge

In [21]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
# progenitor_list[0][2]

# 4. Mating
For each pair of  parents we'll generate an offspring pair. Since we cannot repeat cities what we'll do is copy a random chunk from one progenitor and fill the blanks with the other progenitor.

In [22]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
# new_population_set[0]

# 5. Mutation
Now for each element of the new population we add a random chance of swapping

In [23]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
# mutated_pop[0]

# 6. Stopping
To select the stopping criteria we'll need to create a loop to stop first. Then I'll set it to loop at 1000 iterations.

In [24]:
best_solution = [-1,np.inf,np.array([])]
for i in range(1000):
    if i%100==0: 
        print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)

0 25.365555512197435 34.66498638617748 14/01/23 20:19
100 26.398155552425152 34.37586500908275 14/01/23 20:19
200 27.45493502554148 34.596850117415954 14/01/23 20:20
300 27.787866981552554 35.09985367305359 14/01/23 20:20
400 25.348830881573242 34.759764424404224 14/01/23 20:20
500 26.015096641683673 34.53993227118854 14/01/23 20:20
600 25.620995025192332 34.27753055221439 14/01/23 20:20
700 27.625584957459576 35.47259773759919 14/01/23 20:20
800 27.928395146557005 35.05621283752867 14/01/23 20:20
900 25.090435913977437 34.36714607865234 14/01/23 20:20


In [25]:
best_solution

[580,
 18.075626826443454,
 array([['38 Rue Balard', '4 Boulevard de Grenelle',
         '124 Rue du Faubourg Saint-Honore',
         '2 Rue du Faubourg Saint-Honore', '5 Rue Audran',
         '43 Avenue de Clichy', '31 Avenue Secretan', '24 Rue Tiquetonne',
         '85 Boulevard de Sebastopol', "30 Avenue d'Italie"]], dtype=object)]

In [26]:
best_road = best_solution[2][0]
best_road

array(['38 Rue Balard', '4 Boulevard de Grenelle',
       '124 Rue du Faubourg Saint-Honore',
       '2 Rue du Faubourg Saint-Honore', '5 Rue Audran',
       '43 Avenue de Clichy', '31 Avenue Secretan', '24 Rue Tiquetonne',
       '85 Boulevard de Sebastopol', "30 Avenue d'Italie"], dtype=object)

In [27]:
cities_dict[best_road[0]]

[48.8428844, 2.2773907]

In [28]:
m = folium.Map(location=cities_dict[best_road[0]], zoom_start=12, tiles="Stamen Terrain")
loc = []
for el in best_road:
  loc.append(cities_dict[el])
  folium.Marker(
      cities_dict[el], popup=f"<i>{el}</i>"
  ).add_to(m)
loc.append(cities_dict[best_road[0]])
folium.PolyLine(loc,
                color='blue',
                weight=4,
                opacity=0.9).add_to(m)
m
#m.save("heat_map.html")

In [2]:
import sys
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
class window(QWidget):
   def __init__(self, parent = None):
      super(window, self).__init__(parent)
      self.resize(1800,800)
      self.setWindowTitle("PyQt5")
      self.label = QLabel(self)
      self.label.setText("Hello World")
      font = QFont()
      font.setFamily("Arial")
      font.setPointSize(18)
      self.label.setFont(font)
      self.label.move(50,20)
def main():
    app = QApplication(sys.argv)
    ex = window()
    ex.show()
    try:
       sys.exit(app.exec_())
    except:
        pass
if __name__ == '__main__':
   main()

In [32]:
m.foo()

AttributeError: 'Map' object has no attribute 'foo'